In [283]:
import pandas as pd
import re
import string
import unidecode
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

In [284]:
nltk.download('punkt')
nltk.download('stopwords')
stemmer = SnowballStemmer('spanish')
pd.set_option('max_colwidth', 1000)

[nltk_data] Downloading package punkt to /Users/fernandoa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fernandoa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [285]:
tweets = pd.read_csv('assets/clean_liked_tuits.csv', encoding='utf-8', sep=',')
# We just select textual content for the moment. Other columns shall be used later on
tweets = tweets[['Text', 'Candidate']].copy()
initial_size = len(tweets.index)
tweets.dropna(inplace=True)
tweets.rename({'Candidate': 'Label'}, axis='columns', inplace=True)
tweets['Label'] = tweets['Label'].map({'Petro': 'left', 'Fajardo': 'center', 'Uribe': 'right'})

In [286]:
tweets.head()

,Text,Label
0,"En el gobierno de \n@petrogustavo\n se eliminara el servicio Militar obligatorio y se creará el servicio Militar social voluntario, priorizando la capacitación y la educación de los jóvenes que ingresén, el reclutamiento forzado por parte del estado para los más pobres quedará atrás",left
1,"Cada quien es responsable de sus decisiones, más aún cuando se trata del futuro de 50 millones de colombianos. \n@OIZuluaga\n hizo un llamado a la unidad, \n@FicoGutierrez\n @davidbarguil\n lo entendieron, lastima que a su compañeros de coalición les pudo más su propio interés!",left
2,"La campaña de \n@petrogustavo\n hizo levantar al liberalismo antioqueño para trabajar y soñar con el cambio y dejar atrás el régimen de la corrupción.\n\n¡Estamos haciendo historia, estamos conquistando corazones!\n#ElPactoEsBello \n#PetroEnLaEstrella",left
3,Por Dios santo \n¡NO MÁS!,left
4,"Rechazamos los actos vandálicos contra la sede del Pacto Histórico. Este es un espacio para las juventudes, los comerciantes, las mujeres y cualquier persona que desee un cambio en Colombia.\n\nNo podemos permitir la violencia en ninguna de sus formas.",left


In [287]:
diff = initial_size - len(tweets.index)
print(f'{diff} records were lost; that is {diff/initial_size}% of the dataset.')

4 records were lost; that is 0.002% of the dataset.


In [288]:
def depure(text):
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text) # Remove links and url's
    text = re.sub('#\S+', ' ', text) # Remove hashtags
    text = re.sub('@\S+', ' ', text) # Remove Twitter mentions
    text = re.sub('\w*\d\w*', ' ', text) # Remove words with numbers
    text = unidecode.unidecode(text) # More importantly, remove accents
    text = re.sub(r'[^\w\s]', ' ', text) # Remove puntuaction
    text = re.sub('\n', ' ', text) # Remove filthy, inconvinient new lines >:)))
    return text

In [289]:
spanish_stopwords = nltk.corpus.stopwords.words('spanish')
def remove_stopwords(words):
    return [word for word in words if word not in spanish_stopwords]

In [290]:
def stem(words):
    return [stemmer.stem(word) for word in words]

In [291]:
def process(text):
    depured_text = depure(text)
    tokens = word_tokenize(depured_text)
    non_stopwords = remove_stopwords(tokens)
    stemmed = stem(non_stopwords)
    return stemmed

In [292]:
tweets['Text'] = tweets['Text'].apply(process)

In [293]:
tweets.head()

,Text,Label
0,"[gobiern, elimin, servici, milit, obligatori, cre, servici, milit, social, voluntari, prioriz, capacitacion, educacion, joven, ingres, reclut, forz, part, mas, pobr, qued, atras]",left
1,"[cad, respons, decision, mas, aun, trat, futur, millon, colombian, hiz, llam, unid, entend, lastim, companer, coalicion, pud, mas, propi, inter]",left
2,"[campan, hiz, levant, liberal, antioquen, trabaj, son, cambi, dej, atras, regim, corrupcion, hac, histori, conquist, corazon]",left
3,"[dios, sant, mas]",left
4,"[rechaz, actos, vandal, sed, pact, histor, espaci, juventud, comerci, mujer, cualqui, person, dese, cambi, colombi, pod, permit, violenci, ningun, form]",left
